created May 2023, by Dirk Derichsweiler
# Import an 3th Party Application

this notebook describes how to import an 3th party application into Ezmeral Unfified Application

create blueprint:
```bash
helm create techhours
```

add the following to your Charts.yaml and adapt it to your needs:

**required:**
- endpoint value

```bash
#Platform related options
ezua:
  oidc:
    client_id: "${OIDC_CLIENT_ID}"
    client_secret: "${OIDC_CLIENT_SECRET}"
    domain: "${OIDC_DOMAIN}"

  domainName: "${DOMAIN_NAME}"
  #Use next options in order to configure the application endpoint.
  #Example of a VirtualService is here:
  virtualService:
    endpoint: "techhours.${DOMAIN_NAME}"
    istioGateway: "istio-system/ezaf-gateway"

  authorizationPolicy:
    namespace: "istio-system"
    providerName: "oauth2-proxy"
    matchLabels:
      istio: "ingressgateway"

```

add the following files to your template directory:

- authpolicy.yaml
- kyverno-cluster-policy.yaml   -> exchange with you helm-chart name  "  name: add-vendor-app-labels-{{ include "techhours.fullname" . }}"
- virtualService.yaml -> exchange with you helm-chart name

```bash
❯ cat kyverno-cluster-policy.yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: add-vendor-app-labels-{{ include "techhours.fullname" . }}
  annotations:
    "helm.sh/hook": pre-install
    "helm.sh/hook-delete-policy": before-hook-creation
spec:
  background: false
  rules:
  - name: add-vendor-app-labels
    match:
      any:
      - resources:
          namespaces:
          - {{ .Release.Namespace }}
          kinds:
          - Pod
    mutate:
      patchStrategicMerge:
        metadata:
          labels:
            hpe-ezua/type: vendor-service

```

adapt to your needs: ***(required)***

```bash
❯ cat virtualService.yaml
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {{ include "techhours.fullname" . }}
  labels:
    {{- include "techhours.labels" . | nindent 4 }}
spec:
  gateways:
    - {{ .Values.ezua.virtualService.istioGateway }}
  hosts:
    - {{ .Values.ezua.virtualService.endpoint | required "\nValid .Values.ezua.virtualService.endpoint is required !" }}
  http:
    - match:
        - uri:
            prefix: /
      rewrite:
        uri: /
      route:
        - destination:
            host: {{ include "techhours.fullname" . }}.{{ .Release.Namespace }}.svc.cluster.local
            port:
              number: {{ .Values.service.port }}

```

adapt to your needs: ***(required)***

```bash
apiVersion: security.istio.io/v1beta1
kind: AuthorizationPolicy
metadata:
  name: {{ .Release.Name }}-auth-policy
  namespace: {{ .Values.ezua.authorizationPolicy.namespace }}
spec:
  action: CUSTOM
  provider:
    name: {{ .Values.ezua.authorizationPolicy.providerName }}
  rules:
    - to:
        - operation:
            hosts:
            - {{ .Values.ezua.virtualService.endpoint }}
  selector:
    {{- with .Values.ezua.authorizationPolicy.matchLabels }}
    matchLabels:
      {{- toYaml . | nindent 6 }}
    {{- end }}

```

execute:
```bash
helm package techhours

```

# Login to HPE Ezmeral Unified Analytics Software as admin user

this information is required to import the application:

```bash
❯ cat Chart.yaml
apiVersion: v2
name: techhours
description: A Helm chart for Kubernetes

# A chart can be either an 'application' or a 'library' chart.
#
# Application charts are a collection of templates that can be packaged into versioned archives
# to be deployed.
#
# Library charts provide useful utilities or functions for the chart developer. They're included as
# a dependency of application charts to inject those utilities and functions into the rendering
# pipeline. Library charts do not define any templates and therefore cannot be deployed.
type: application

# This is the chart version. This version number should be incremented each time you make changes
# to the chart and its templates, including the app version.
# Versions are expected to follow Semantic Versioning (https://semver.org/)
version: 0.1.0

# This is the version number of the application being deployed. This version number should be
# incremented each time you make changes to the application. Versions are not expected to
# follow Semantic Versioning. They should reflect the version the application is using.
# It is recommended to use it with quotes.
appVersion: "1.16.0"

```

- enter the value "version" from you Chart.yaml file into the Helm Chart Version Messagebox.
- enter the value "appVersion" into the Version Messagebox.
- the value name: enter into the Helm Chart Name.

<img src="./media/import_application.png" style="height:800px">